# MD simulation of YYY peptide

Let's run a MD simulation of your peptide! Just like before, you'll load in the force field parameters, initial coordinates (3D structure), and topology. Then you'll subject this system to energy minimization, followed by a short equilibration simulation and a longer production simulation (for calculating statistics).

In [ ]:
from openmm import app
import openmm as mm
from simtk import unit
from sys import stdout
import time as time

In [ ]:
# read in files
prmtop = app.AmberPrmtopFile('YYY.prmtop') # change XXX to your peptide's name
inpcrd = app.AmberInpcrdFile('YYY.inpcrd') # same here!

The code below sets up our system for simulation. Notice that the improved Generalized Born implicit solvent model is used instead of explicitly solvating molecule. This should speed up our computation.

In [ ]:
# OpenMM setup
system = prmtop.createSystem(nonbondedMethod=app.NoCutoff, constraints=app.HBonds, implicitSolvent=app.GBn2)
temperature = 298.15*unit.kelvin
integrator = mm.LangevinIntegrator(temperature, 1/unit.picosecond, 2*unit.femtoseconds)
platform = mm.Platform.getPlatformByName('CUDA')
simulation = app.Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)

Next we can subject the system to 100 steps of energy minimization. 

In [ ]:
# minimization
st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

print('Minimizing...')
simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

After energy minimization, we can run a short equilibration simulation.

In [ ]:
# equilibration
simulation.context.setVelocitiesToTemperature(298.15*unit.kelvin)
print('Equilibrating...')
tinit=time.time()
simulation.step(50000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Finally, let's run the production simulation. Note that we write the atomic coordinates out to a **trajectory file** (in this case: `YYY_sim.dcd`) every 500 time steps. We don't do this every simulation step because otherwise the file size would become prohibitively large.

Make sure to change the `YYY` to your peptide.

In [ ]:
# production
simulation.reporters.append(app.DCDReporter('YYY_sim.dcd', 500)) # change YYY to your peptide's name
simulation.reporters.append(app.StateDataReporter(stdout, 50000, step=True, time=True,
    potentialEnergy=True, temperature=True, speed=True, separator='\t'))

In [ ]:
print('Running Production...')
simulation.step(500000) 
print('Done!')